In [4]:
import pandas as pd
df = pd.read_csv("RaceTimes.csv")
test = df['Lap Time Seconds'].mean()
print(test)

45.237519999999996


In [5]:
Q1 = df['Lap Time Seconds'].quantile(0.25)
Q3 = df['Lap Time Seconds'].quantile(0.75)
IQR = Q3 - Q1

# Filter out the outliers
df_filtered = df[~((df['Lap Time Seconds'] < (Q1 - 1.5 * IQR)) | (df['Lap Time Seconds'] > (Q3 + 1.5 * IQR)))]

test = df_filtered['Lap Time Seconds'].mean()
print(test)


40.258976973684206


In [6]:
df_filtered.to_csv('raceTimes.csv', index=False)